In [4]:
import os
import hjson
import string
from openpyxl import Workbook, load_workbook


In [5]:
def is_ascii(s):
    return all(c in string.printable for c in s)

def extract_ascii_displaynames(input_folder, filter_string, result_file):
    if not os.path.exists(result_file):
        # Create a new Excel file with a header row
        wb = Workbook()
        ws = wb.active
        wb.save(result_file)
    else:
        # Open the existing Excel file
        wb = load_workbook(result_file)
        ws = wb.active

    for filename in os.listdir(input_folder):
        if filename.endswith(".hjson") and filter_string in filename:
            file_path = os.path.join(input_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = hjson.load(f)
                except Exception as e:
                    print(f"Failed to parse {filename}: {e}")
                    continue

                for key, value in data.items():
                    try:
                        display_name = value.get("DisplayName", "")
                        if isinstance(display_name, str) and display_name != "" and is_ascii(display_name) and not '{' in display_name and not '.' in display_name and not all(c.isupper() for c in display_name if c.isalpha()):
                            if not any(ws.cell(row=i, column=1).value == display_name for i in range(1, ws.max_row + 1)):
                                ws.append([display_name])
                    except Exception as e:
                        continue
        wb.save(result_file)
        


In [6]:
if __name__ == "__main__":
    input_folder = "../Localization/zh-Hans"
    filter_string = "Items"
    output_folder = "Output/untranslated.xlsx"
    extract_ascii_displaynames(input_folder, filter_string, output_folder)